### 描述性统计分析可视化

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import random
%matplotlib inline

In [3]:
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20

In [4]:
plt.rcParams['font.sans-serif']='SimHei' #解决中文乱码问题
plt.rcParams['axes.unicode_minus']=False #解决负号无法显示的问题
%config InlineBackend.figure_format = 'svg' #将图表设置为svg格式以保证更为清晰的显示

In [5]:
def func(df, column='月度销售'):
    return df[column].cumsum()

In [6]:
f = lambda x:x.format(x*100,'0.2f')

In [7]:
# 加载原始数据
df0 = pd.read_excel(r'C:\Users\Nikola\Desktop\Python\销售记录.xlsx',
                    encoding='utf8',
                    index_col='下单日期')

In [8]:
# 基础数据获取
df1 = df0[['订单编号', '客户ID','客户省份', '销售代表ID','产品ID', '产品名称', '数量', '单价', '金额']].sort_index()

In [9]:
df1['金额'] = df1['金额']/10000

In [10]:
df2 = df1.groupby(df1.index.to_period('M'))[['金额']].sum()
df2.rename(columns={'金额':'月度销售'},inplace=True)
df2['上月销售'] = df2.shift(1)['月度销售']
df2['去年同月'] = df2.shift(12)['月度销售']
df2['MOM环比'] = (df2['月度销售']/df2['上月销售']-1)
df2['YOY同比'] = df2['月度销售']/df2['去年同月']-1

In [11]:
df3 = pd.DataFrame(df2.groupby(df2.index.year).apply(func))
df3.reset_index(level=0,inplace=True)
df3.drop('下单日期',axis=1,inplace=True)
df3.columns=['年度累计']
df3['去年累计'] = df3.shift(12)['年度累计']

In [12]:
DF = df2.join(df3)
DF['累计同比'] = DF['年度累计']/DF['去年累计']-1
DF.reindex(columns=['月度销售', '上月销售', '去年同月', '去年累计','年度累计', 'MOM环比', 'YOY同比',  '累计同比'])

,月度销售,上月销售,去年同月,去年累计,年度累计,MOM环比,YOY同比,累计同比
下单日期,,,,,,,,
2017-01,397.063643,NaN,NaN,NaN,397.063643,NaN,NaN,NaN
2017-02,147.553836,397.063643,NaN,NaN,544.617479,-0.628387,NaN,NaN
2017-03,297.635327,147.553836,NaN,NaN,842.252806,1.017130,NaN,NaN
2017-04,177.004740,297.635327,NaN,NaN,1019.257545,-0.405297,NaN,NaN
2017-05,309.248187,177.004740,NaN,NaN,1328.505733,0.747118,NaN,NaN
2017-06,411.408925,309.248187,NaN,NaN,1739.914658,0.330352,NaN,NaN
2017-07,342.725240,411.408925,NaN,NaN,2082.639898,-0.166947,NaN,NaN
2017-08,217.766675,342.725240,NaN,NaN,2300.406574,-0.364603,NaN,NaN
2017-09,345.534238,217.766675,NaN,NaN,2645.940812,0.586718,NaN,NaN


In [13]:
DF.to_excel(r'C:\Users\Nikola\Desktop\1-1 同比环比.xlsx')

In [14]:
# 数据透视行月份，列年份，值销售金额
df1.index = df1.index.to_period('D')
df1

,订单编号,客户ID,客户省份,销售代表ID,产品ID,产品名称,数量,单价,金额
下单日期,,,,,,,,,
2017-01-01,SO45308,29861,安徽,201,743,山地自行车,5,2039.9940,1.019997
2017-01-01,SO45304,29664,湖南,210,743,山地自行车,2,2039.9940,0.407999
2017-01-01,SO45304,29664,湖南,210,743,山地自行车,1,2039.9940,0.203999
2017-01-01,SO45304,29664,湖南,210,743,山地自行车,2,2039.9940,0.407999
2017-01-01,SO45272,29539,北京,205,709,公路自行车,3,419.4589,0.125838
2017-01-01,SO45272,29539,北京,205,709,公路自行车,1,419.4589,0.041946
2017-01-01,SO45272,29539,北京,205,709,公路自行车,4,419.4589,0.167784
2017-01-01,SO45304,29664,湖南,210,743,山地自行车,2,2024.9940,0.404999
2017-01-01,SO45272,29539,北京,205,709,公路自行车,1,419.4589,0.041946


In [15]:
df4 = df1.pivot_table(values='金额',index=df1.index.month, columns=df1.index.year,aggfunc='sum')
df4

下单日期,2017,2018
下单日期,,
1,397.063643,208.821842
2,147.553836,232.008428
3,297.635327,341.396376
4,177.004740,253.465075
5,309.248187,330.915627
6,411.408925,517.468268
7,342.725240,500.141280
8,217.766675,334.506757
9,345.534238,454.021220


In [16]:
df4.columns.name = '年份'
df4.index.name ='月份'
df4['同比增长'] = df4.iloc[:,1]/df4.iloc[:,0]-1
df4

年份,2017,2018,同比增长
月份,,,
1,397.063643,208.821842,-0.474085
2,147.553836,232.008428,0.572365
3,297.635327,341.396376,0.147029
4,177.004740,253.465075,0.431968
5,309.248187,330.915627,0.070065
6,411.408925,517.468268,0.257795
7,342.725240,500.141280,0.459307
8,217.766675,334.506757,0.536079
9,345.534238,454.021220,0.313969


In [17]:
df4.to_excel(r'C:\Users\Nikola\Desktop\1-2 同比环比.xlsx')

In [18]:
# 季度同比环比
df5 = df1.resample('Q')[['金额']].sum()
df5['上季销售'] = df5.shift(1)['金额']
df5['同季销售'] = df5.shift(4)['金额']
df5.columns = ['季度销售','上季销售','同季销售']
df5['同比YOY'] = df5.季度销售/df5.同季销售-1
df5['环比MoM'] = df5.季度销售/df5.上季销售-1
df5

,季度销售,上季销售,同季销售,同比YOY,环比MoM
下单日期,,,,,
2017Q1,842.252806,NaN,NaN,NaN,NaN
2017Q2,897.661852,842.252806,NaN,NaN,0.065787
2017Q3,906.026154,897.661852,NaN,NaN,0.009318
2017Q4,725.148882,906.026154,NaN,NaN,-0.199638
2018Q1,782.226646,725.148882,842.252806,-0.071269,0.078712
2018Q2,1101.848970,782.226646,897.661852,0.227466,0.408606
2018Q3,1288.669256,1101.848970,906.026154,0.422331,0.169552
2018Q4,1219.141844,1288.669256,725.148882,0.681230,-0.053953


In [19]:
df5.环比MoM = df5['环比MoM'].apply(lambda x: '%.2f%%' % (x*100))
df5.同比YOY = df5['同比YOY'].apply(lambda x: '%.2f%%' % (x*100))
df5.to_excel(r'C:\Users\Nikola\Desktop\1-3 同比环比.xlsx')

In [40]:
# 创建时间序列表
ts = pd.date_range('20170101','20181231',freq='D')
df6 = pd.DataFrame(np.random.rand(len(ts)),index=ts,columns=['date'])
df6['年'] = df6.index.year
df6['季'] = df6.index.quarter
df6['月'] = df6.index.month
df6['周'] = df6.index.weekofyear
df6['星期'] = df6.index.weekday+1
df6['类别'] = np.where(df6.index.weekday<5,'工作日','周末')

df6['年份'] = df6['年'].apply(lambda x:str(x)+'年')
df6['季度'] = df6['季'].apply(lambda x:str(x)+'季度')
df6['月份'] = df6['月'].apply(lambda x:str(x)+'月')
df6['周序'] = df6['周'].apply(lambda x:str(x)+'周')
df6['星期几'] = df6.index.weekday_name

df6.iloc[0,[4,10]]=[1,'1周']
df6.iloc[729,10]=['52周']
df6

,date,年,季,月,周,星期,类别,年份,季度,月份,周序,星期几
2017-01-01,0.783380,2017,1,1,1,7,周末,2017年,1季度,1月,1周,Sunday
2017-01-02,0.926100,2017,1,1,1,1,工作日,2017年,1季度,1月,1周,Monday
2017-01-03,0.412058,2017,1,1,1,2,工作日,2017年,1季度,1月,1周,Tuesday
2017-01-04,0.968155,2017,1,1,1,3,工作日,2017年,1季度,1月,1周,Wednesday
2017-01-05,0.863489,2017,1,1,1,4,工作日,2017年,1季度,1月,1周,Thursday
2017-01-06,0.163594,2017,1,1,1,5,工作日,2017年,1季度,1月,1周,Friday
2017-01-07,0.142382,2017,1,1,1,6,周末,2017年,1季度,1月,1周,Saturday
2017-01-08,0.720059,2017,1,1,1,7,周末,2017年,1季度,1月,1周,Sunday
2017-01-09,0.959142,2017,1,1,2,1,工作日,2017年,1季度,1月,2周,Monday
2017-01-10,0.938571,2017,1,1,2,2,工作日,2017年,1季度,1月,2周,Tuesday


In [41]:
# # 创建时间序列表
# ts = pd.date_range('20170101','20181231',freq='D')
# df6 = pd.DataFrame(ts,columns=['date'])
# df6['年'] = df6.date.dt.year
# df6['季'] = df6.date.dt.quarter
# df6['月'] = df6.date.dt.month
# df6['周'] = df6.date.dt.weekofyear
# df6['星期'] = df6.date.dt.weekday+1
# df6['类别'] = np.where(df6.date.dt.weekday<5,'工作日','周末')

# df6['年份'] = df6['年'].apply(lambda x:str(x)+'年')
# df6['季度'] = df6['季'].apply(lambda x:str(x)+'季度')
# df6['月份'] = df6['月'].apply(lambda x:str(x)+'月')
# df6['周序'] = df6['周'].apply(lambda x:str(x)+'周')
# df6['星期几'] = df6.date.dt.weekday_name
# df6.set_index('date',inplace=True)
# df6

,季度销售,上季销售,同季销售,同比YOY,环比MoM
下单日期,,,,,
2017Q1,842.252806,NaN,NaN,nan%,nan%
2017Q2,897.661852,842.252806,NaN,nan%,6.58%
2017Q3,906.026154,897.661852,NaN,nan%,0.93%
2017Q4,725.148882,906.026154,NaN,nan%,-19.96%
2018Q1,782.226646,725.148882,842.252806,-7.13%,7.87%
2018Q2,1101.848970,782.226646,897.661852,22.75%,40.86%
2018Q3,1288.669256,1101.848970,906.026154,42.23%,16.96%
2018Q4,1219.141844,1288.669256,725.148882,68.12%,-5.40%


In [ ]:
df1.index=df1.index.to_timestamp()

In [64]:
df7 = pd.merge(df1,df6,left_index=True,right_index=True,how='left')
df7.index=df7.index.to_period('D')
df7.resample('M')[['金额']].sum()
df7.resample('BM')[['金额']].sum()

ValueError: Invalid frequency: BM